In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError

import re
import os
import pickle

In [5]:
from enum import Enum

In [ ]:
"""
youtube = YouTube()
youtube.authenticate(credentials_file='', token_path='')
youtube = YouTube(credentials_file='', token_path='')
youtube.search_video(): query -> List[YouTubeVideo]
youtube.search_channel(): query -> List[YouTubeChannel]
youtube.search_playlist(): query -> List[PlayList]
youtube.YouTubeVideo(video_id='') -> YouTubeVideo
youtube.YouTubeVideo(video_url='')
youtube.errors.
youtube.utils
"""

In [170]:
class YouTubeAPIConstants:
    TOKEN_FILE = 'token.pickle'
    API_SERVICE_NAME = 'youtube'
    API_VERSION = 'v3'
    SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [190]:
class Authenticate:
    """Handle the YouTube authentication process."""
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    __TOKEN_FILE = YouTubeAPIConstants.TOKEN_FILE
    __API_SERVICE_NAME = YouTubeAPIConstants.API_SERVICE_NAME
    __API_VERSION = YouTubeAPIConstants.API_VERSION
    __SCOPES = YouTubeAPIConstants.SCOPES
    
    def __init__(self, client_secrets_file: str , api_token_path: str = ''):
        """Create the auth object."""
        self.__credentials = None
        
        self.__verify_client_secret_file(client_secrets_file)
        self.__client_secrets_file = client_secrets_file
        
        if not api_token_path or not os.path.exists(api_token_path):
            self.__api_token_path = self.__get_default_api_token_path()
        else:
            self.__api_token_path = api_token_path
            
    def __verify_client_secret_file(self, client_secrets_file: str) -> None:
        """Verfy the client secret file."""
        if not client_secrets_file:
            raise ValueError('The clients secret file path has to be provided.')
        if not isinstance(client_secrets_file, str):
            raise TypeError('The clients secret file should be a string.')
        if not os.path.exists(client_secrets_file):
            raise ValueError(f'The path {client_secrets_file} does not exist!')
        
    def __get_default_api_token_path(self):
        """Generate the default api token file location."""
        current_user_home_dir = os.path.expanduser('~')
        api_token_path = os.path.join(current_user_home_dir, self.__TOKEN_FILE)
        return api_token_path
    
    def __authenticate_youtube(self):
        """Authenticate the YouTube API."""
        if os.path.exists(self.__api_token_path):
            with open(self.__api_token_path, "rb") as token:
                self.__credentials = pickle.load(token)
        # if there are no (valid) credentials availablle, let the user log in.
        if not self.__credentials or not self.__credentials.valid:
            if self.__credentials and self.__credentials.expired and self.__credentials.refresh_token:
                self.__credentials.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.__client_secrets_file, self.__SCOPES)
                self.__credentials = flow.run_local_server(port=0)
            # save the credentials for the next run
            with open(self.__api_token_path, "wb") as token:
                pickle.dump(self.__credentials, token)

        return build(self.__API_SERVICE_NAME, self.__API_VERSION, credentials=self.__credentials)
    
    def authenticate(self):
        try:
            youtube_client =  self.__authenticate_youtube()
        except RefreshError as e:
            raise Exception('The token is expired. Kindly generate a new one.')
        else:
            return youtube_client

In [191]:
class YouTubeSearchQuery:
    """A query to pass to the search resource."""
    
    def __init__(self, query_string: str):
        self.__query_string = query_string
        
    @property
    def query_string(self):
        return self.__query_string
    
    @query_string.setter
    def query_string(self, query_str: str):
        if not query_str:
            raise ValueError('The query string has to be provided')
        if not isinstance(query_str, str):
            raise TypeError('The query string has to be a string')
        self.__query_string = query_str

        
class YouTubeVideoSearchQuery(YouTubeSearchQuery):
    pass

In [192]:
class YouTubeSearchType:
    VIDEO = 'video'
    CHANNEL = 'channel'
    PLAYLIST = 'playlist'

In [193]:
class YouTubeSearch:
    MAX_RESULTS = 10
    REGION_CODE = 'US'
    
    def __init__(self, *args, **kwargs):
        pass

In [194]:
class VideoSearch(YouTubeSearch):
    def __init__(self, query_string: str):
        self.__type = YouTubeSearchType.VIDEO
        self.__query = YouTubeSearchQuery(query_string)
        
    def __get_query(self):
        return self.__query.query_string
        
    def basic_info(self):
        basic_info_params = self.__generate_basic_info_params()
        return basic_info_params
    
    def advanced_info(self):
        pass
    
    def all_info(self):
        pass
    
    def __generate_basic_info_params(self, next_page_token=''):
        basic_info_params = dict(
            part='id,snippet',
            type=self.__type,
            q=self.__get_query(),
            maxResults=self.MAX_RESULTS,
            regionCode=self.REGION_CODE
        ) 
        if next_page_token:
            basic_info_params['pageToken'] = next_page_token
        return basic_info_params
    
    def search_video(self, youtube_client, search_type='basic', next_page_token=''):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params(next_page_token='')
            search_request = youtube_client.search().list(
                **basic_info_params
            )
            search_response = search_request.execute()
        return search_response
    
    @staticmethod
    def parse_basic_response(search_result: dict) -> list[dict[str, str]]:
        video_details = []
        video_results = search_result['items']
        for video_result in video_results:
            video = {}
            video_id = video_result['id']
            video_snippet = video_result['snippet']
            video['id'] = video_id['videoId']
            video['title'] = video_snippet['title']
            video['description'] = video_snippet['description']
            video['channelTitle'] = video_snippet['channelTitle']
            video_details.append(video)
        return video_details

In [195]:
class YouTube:
    def __init__(self, credentials_file='', token_path=''):
        self.__auth = Authenticate(credentials_file, token_path)
        self.__youtube_client = None
        self.__is_authenticated = False
        
    def is_authenticated(self):
        return self.__is_authenticated
        
    def authenticate(self, credentials_file='', token_path=''):
        self.__youtube_client = self.__auth.authenticate()
        self.__is_authenticated = True
        
    def get_youtube(self):
        return self.__youtube_client
        
    def search_video(self, query_string: str) -> list[str]:
        video_search = VideoSearch('python')
        search_results = video_search.search_video(self.__youtube_client)
        parsed_results = video_search.parse_basic_response(search_results)
        return parsed_results
    
    def get_video(self, video_url: str):
        """Get a specific video given the video url."""
        video_id = self.__get_video_id(video_url)
        
    def find_video(self, video: str, by: str = 'url'):
        """Find a video by id."""
        if by:
            if not isinstance(by, str):
                raise ValueError('by must be a string')
            if by not in ['url', 'id']:
                raise ValueError('by must be either id or url')
        if by == 'url':
            return self.__get_video_id(video)
    
    @staticmethod
    def __get_video_id(video_url: str) -> str:
        """Get vdeo ID from video url"""
        if not video_url:
            raise ValueError('The video_ur has to be provided.')
        if not isinstance(video_url, str):
            raise TypeError('Te video_url has to be a string.')
        if '=' not in video_url:
            url_format = 'https://www.youtube.com/watch?v=Dqdu-FsBk0s'
            raise ValueError('Te video_url should be of the format "{url_format}"')
        video_url = video_url.split('=')[1]
        return video_url

In [196]:
youtube = YouTube(credentials_file='/home/lyle/Downloads/client_secret.json', 
                  token_path='')
youtube.authenticate()
youtube_client = youtube.get_youtube()

In [197]:
youtube.search_video('python')

[{'id': '_uQrJ0TkZlc',
  'title': 'Python Tutorial - Python Full Course for Beginners',
  'description': 'Python tutorial - Python full course for beginners - Go from Zero to Hero with Python (includes machine learning & web ...',
  'channelTitle': 'Programming with Mosh'},
 {'id': 'rfscVS0vtbw',
  'title': 'Learn Python - Full Course for Beginners [Tutorial]',
  'description': "This course will give you a full introduction into all of the core concepts in python. Follow along with the videos and you'll be a ...",
  'channelTitle': 'freeCodeCamp.org'},
 {'id': 'kqtD5dpn9C8',
  'title': 'Python for Beginners - Learn Python in 1 Hour',
  'description': 'This Python tutorial for beginners show how to get started with Python quickly. Learn to code in 1 hour! Watch this tutorial get ...',
  'channelTitle': 'Programming with Mosh'},
 {'id': 'b093aqAZiPU',
  'title': '👩\u200d💻 Python for Beginners Tutorial',
  'description': 'In this step-by-step Python for beginners tutorial, learn how you c

In [198]:
search_request = youtube_client.videos().list(
    id='rfscVS0vtbw',
    part='snippet,contentDetails,statistics'
)
search_response = search_request.execute()

In [199]:
class FindVideo:
    def __init__(self, video_id: str):
        """Find the video with the given id."""
        self.__video_id = video_id
        
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            id=self.__video_id,
            part='snippet,contentDetails,statistics'
        ) 
        return basic_info_params
    
    def find_video(self, youtube_client):
        """Find the video."""
        basic_info_params = self.__generate_basic_info_params()
        search_request = youtube_client.videos().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        parsed_response = self.__parse_video_details(search_response)
        return parsed_response
    
    def __parse_video_details(self, video_details: dict):
        """Parse the video details.

        Returns
        -------
        parsed_video_details: dict
            A dictionary of the YouTube video details.
        """
        parsed_video_details = dict()
        items = video_details['items'][0]
        parsed_video_details['details'] = dict()
        parsed_video_details['statistics'] = dict()
        parsed_video_details['details']['id'] = items['id']
        parsed_video_details['details']['channelId'] = items['snippet']['channelId']
        parsed_video_details['details']['title'] = items['snippet']['title']
        parsed_video_details['details']['channelTitle'] = items['snippet']['channelTitle']
        parsed_video_details['details']['description'] = items['snippet']['description']
        parsed_video_details['details']['tags'] = items['snippet']['tags']
        parsed_video_details['details']['duration'] = items['contentDetails']['duration']
        parsed_video_details['details']['licensedContent'] = items['contentDetails']['licensedContent']
        parsed_video_details['statistics']['viewCount'] = items['statistics']['viewCount']
        parsed_video_details['statistics']['likeCount'] = items['statistics']['likeCount']
        parsed_video_details['statistics']['commentCount'] = items['statistics']['commentCount']
        return parsed_video_details

In [200]:
find_video = FindVideo(video_id='rfscVS0vtbw')
video_details = find_video.find_video(youtube_client)

In [201]:
video_details

{'details': {'id': 'rfscVS0vtbw',
  'channelId': 'UC8butISFwT-Wl7EV0hUK0BQ',
  'title': 'Learn Python - Full Course for Beginners [Tutorial]',
  'channelTitle': 'freeCodeCamp.org',
  'description': "This course will give you a full introduction into all of the core concepts in python. Follow along with the videos and you'll be a python programmer in no time!\nWant more from Mike? He's starting a coding RPG/Bootcamp - https://simulator.dev/\n\n⭐️ Contents ⭐\n⌨️ (0:00) Introduction\n⌨️ (1:45) Installing Python & PyCharm\n⌨️ (6:40) Setup & Hello World\n⌨️ (10:23) Drawing a Shape\n⌨️ (15:06) Variables & Data Types\n⌨️ (27:03) Working With Strings\n⌨️ (38:18) Working With Numbers\n⌨️ (48:26) Getting Input From Users\n⌨️ (52:37) Building a Basic Calculator\n⌨️ (58:27) Mad Libs Game\n⌨️ (1:03:10) Lists\n⌨️ (1:10:44) List Functions\n⌨️ (1:18:57) Tuples\n⌨️ (1:24:15) Functions\n⌨️ (1:34:11) Return Statement\n⌨️ (1:40:06) If Statements\n⌨️ (1:54:07) If Statements & Comparisons\n⌨️ (2:00:37) Buil

In [202]:
parse_video_details(video_details)

KeyError: 'items'

In [203]:
def parse_video_details(video_details: dict):
    """Parse the video details.
    
    Returns
    -------
    parsed_video_details: dict
        A dictionary of the YouTube video details.
    """
    parsed_video_details = dict()
    items = video_details['items'][0]
    parsed_video_details['id'] = items['id']
    parsed_video_details['channelId'] = items['snippet']['channelId']
    parsed_video_details['title'] = items['snippet']['title']
    parsed_video_details['channelTitle'] = items['snippet']['channelTitle']
    parsed_video_details['description'] = items['snippet']['description']
    parsed_video_details['tags'] = items['snippet']['tags']
    parsed_video_details['duration'] = items['contentDetails']['duration']
    parsed_video_details['licensedContent'] = items['contentDetails']['licensedContent']
    parsed_video_details['viewCount'] = items['statistics']['viewCount']
    parsed_video_details['likeCount'] = items['statistics']['likeCount']
    parsed_video_details['commentCount'] = items['statistics']['commentCount']
    return parsed_video_details

In [204]:
parse_video_details(video_details)

KeyError: 'items'

In [205]:
find_video = FindVideo(video_id='rfscVS0vtbw')
video_details = find_video.find_video(youtube_client)

In [206]:
video_details

{'details': {'id': 'rfscVS0vtbw',
  'channelId': 'UC8butISFwT-Wl7EV0hUK0BQ',
  'title': 'Learn Python - Full Course for Beginners [Tutorial]',
  'channelTitle': 'freeCodeCamp.org',
  'description': "This course will give you a full introduction into all of the core concepts in python. Follow along with the videos and you'll be a python programmer in no time!\nWant more from Mike? He's starting a coding RPG/Bootcamp - https://simulator.dev/\n\n⭐️ Contents ⭐\n⌨️ (0:00) Introduction\n⌨️ (1:45) Installing Python & PyCharm\n⌨️ (6:40) Setup & Hello World\n⌨️ (10:23) Drawing a Shape\n⌨️ (15:06) Variables & Data Types\n⌨️ (27:03) Working With Strings\n⌨️ (38:18) Working With Numbers\n⌨️ (48:26) Getting Input From Users\n⌨️ (52:37) Building a Basic Calculator\n⌨️ (58:27) Mad Libs Game\n⌨️ (1:03:10) Lists\n⌨️ (1:10:44) List Functions\n⌨️ (1:18:57) Tuples\n⌨️ (1:24:15) Functions\n⌨️ (1:34:11) Return Statement\n⌨️ (1:40:06) If Statements\n⌨️ (1:54:07) If Statements & Comparisons\n⌨️ (2:00:37) Buil

In [207]:
find_video = FindVideo(video_id='x7X9w_GIm1s')
video_details = find_video.find_video(youtube_client)
video_details

{'details': {'id': 'x7X9w_GIm1s',
  'channelId': 'UCsBjURrPoezykLs9EqgamOA',
  'title': 'Python in 100 Seconds',
  'channelTitle': 'Fireship',
  'description': "Python is arguably the world's most popular programming language. It is easy to learn, yet suitable in professional software like web applications, data science, and server-side scripts. https://fireship.io/tags/python/\n\n#python #programming #100SecondsOfCode\n\n🔗 Resources\n\nPython Docs https://docs.python.org/3/\nPython TIOBE Ranking https://www.infoworld.com/article/3636789/python-tops-tiobe-language-index.html\n\n🔥 Get More Content - Upgrade to PRO\n\nUpgrade to Fireship PRO at https://fireship.io/pro\nUse code lORhwXd2 for 25% off your first payment. \n\n🎨 My Editor Settings\n\n- Atom One Dark \n- vscode-icons\n- Fira Code Font\n\n🔖 Topics Covered\n\n- Is YouTube built with Python?\n- Is Python Strongly typed?\n- Python syntax and indentation rules\n- When was python released?",
  'tags': ['webdev', 'app development', '

In [208]:
class YouTubeVideoStats:
    def __init__(self, viewCount: int, likeCount: int, commentCount: int):
        self.__view_count = int(viewCount)
        self.__like_count = int(likeCount)
        self.__comment_count = int(commentCount)
        
    def get_video_stats(self):
        video_stats = {
            'view_count': self.__view_count,
            'like_count': self.__like_count,
            'comment_count': self.__comment_count
        }
        return video_stats

In [209]:
def create_video_stats(video_details):
    stats = YouTubeVideoStats(**video_details['statistics'])
    return stats

In [210]:
stats = create_video_stats(video_details)
stats.get_video_stats()

{'view_count': 1680645, 'like_count': 76703, 'comment_count': 1438}

In [211]:
class YouTubeVideoDetails:
    def __init__(self, id: str, channelId: str, title: str, channelTitle: str, 
                 description: str, tags: list[str], duration: str, licensedContent: bool):
        self.__id = id
        self.__channel_id = channelId
        self.__title = title
        self.__channel_title = channelTitle
        self.__description = description
        self.__tags = tags
        self.__duration = duration
        self.__licensed_content = licensedContent
        
    def get_video_details(self):
        video_details = {
            'id': self.__id,
            'channel_id': self.__channel_id,
            'title': self.__title,
            'channel_title': self.__channel_title,
            'description': self.__description,
            'tags': self.__tags,
            'duration': self.__duration,
            'licensed_content': self.__licensed_content
        }
        return video_details

In [212]:
def create_video_details(video_details):
    details = YouTubeVideoDetails(**video_details['details'])
    return details

In [213]:
details = create_video_details(video_details)
details.get_video_details()

{'id': 'x7X9w_GIm1s',
 'channel_id': 'UCsBjURrPoezykLs9EqgamOA',
 'title': 'Python in 100 Seconds',
 'channel_title': 'Fireship',
 'description': "Python is arguably the world's most popular programming language. It is easy to learn, yet suitable in professional software like web applications, data science, and server-side scripts. https://fireship.io/tags/python/\n\n#python #programming #100SecondsOfCode\n\n🔗 Resources\n\nPython Docs https://docs.python.org/3/\nPython TIOBE Ranking https://www.infoworld.com/article/3636789/python-tops-tiobe-language-index.html\n\n🔥 Get More Content - Upgrade to PRO\n\nUpgrade to Fireship PRO at https://fireship.io/pro\nUse code lORhwXd2 for 25% off your first payment. \n\n🎨 My Editor Settings\n\n- Atom One Dark \n- vscode-icons\n- Fira Code Font\n\n🔖 Topics Covered\n\n- Is YouTube built with Python?\n- Is Python Strongly typed?\n- Python syntax and indentation rules\n- When was python released?",
 'tags': ['webdev', 'app development', 'lesson', 'tutor

In [214]:
class YouTubeVideo:
    """A YouTube Video."""
    def __init__(self, video_details):
        self.__video_stats = self.__create_video_stats(video_details)
        self.__video_details = self.__create_video_details(video_details)
        self.__video_top_level_comments = None
        
    def get_video_stats_details(self):
        video_stats_details = dict()
        video_stats_details['details'] = self.get_video_details()
        video_stats_details['statistics'] = self.get_video_stats()
        return video_stats_details
        
    def __create_video_stats(self, video_details: dict):
        video_stats = YouTubeVideoStats(**video_details['statistics'])
        return video_stats
    
    def __create_video_details(self, video_details: dict):
        video_details = YouTubeVideoDetails(**video_details['details'])
        return video_details
        
    def get_video_stats(self):
        return self.__video_stats.get_video_stats()
    
    def get_video_details(self):
        return self.__video_details.get_video_details()
    
    def get_video_top_level_comments(self):
        pass

In [215]:
youtube_video = YouTubeVideo(video_details)

In [216]:
youtube_video.get_video_stats()

{'view_count': 1680645, 'like_count': 76703, 'comment_count': 1438}

In [217]:
youtube_video.get_video_details()

{'id': 'x7X9w_GIm1s',
 'channel_id': 'UCsBjURrPoezykLs9EqgamOA',
 'title': 'Python in 100 Seconds',
 'channel_title': 'Fireship',
 'description': "Python is arguably the world's most popular programming language. It is easy to learn, yet suitable in professional software like web applications, data science, and server-side scripts. https://fireship.io/tags/python/\n\n#python #programming #100SecondsOfCode\n\n🔗 Resources\n\nPython Docs https://docs.python.org/3/\nPython TIOBE Ranking https://www.infoworld.com/article/3636789/python-tops-tiobe-language-index.html\n\n🔥 Get More Content - Upgrade to PRO\n\nUpgrade to Fireship PRO at https://fireship.io/pro\nUse code lORhwXd2 for 25% off your first payment. \n\n🎨 My Editor Settings\n\n- Atom One Dark \n- vscode-icons\n- Fira Code Font\n\n🔖 Topics Covered\n\n- Is YouTube built with Python?\n- Is Python Strongly typed?\n- Python syntax and indentation rules\n- When was python released?",
 'tags': ['webdev', 'app development', 'lesson', 'tutor

In [218]:
class FindVideo:
    def __init__(self, video_id: str):
        """Find the video with the given id."""
        self.__video_id = video_id
        
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            id=self.__video_id,
            part='snippet,contentDetails,statistics'
        ) 
        return basic_info_params
    
    def find_video(self, youtube_client):
        """Find the video."""
        basic_info_params = self.__generate_basic_info_params()
        search_request = youtube_client.videos().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        parsed_response = self.__parse_video_details(search_response)
        youtube_video = YouTubeVideo(parsed_response)
        return youtube_video
    
    def __parse_video_details(self, video_details: dict):
        """Parse the video details.

        Returns
        -------
        parsed_video_details: dict
            A dictionary of the YouTube video details.
        """
        parsed_video_details = dict()
        items = video_details['items'][0]
        parsed_video_details['details'] = dict()
        parsed_video_details['statistics'] = dict()
        parsed_video_details['details']['id'] = items['id']
        parsed_video_details['details']['channelId'] = items['snippet']['channelId']
        parsed_video_details['details']['title'] = items['snippet']['title']
        parsed_video_details['details']['channelTitle'] = items['snippet']['channelTitle']
        parsed_video_details['details']['description'] = items['snippet']['description']
        parsed_video_details['details']['tags'] = items['snippet']['tags']
        parsed_video_details['details']['duration'] = items['contentDetails']['duration']
        parsed_video_details['details']['licensedContent'] = items['contentDetails']['licensedContent']
        parsed_video_details['statistics']['viewCount'] = items['statistics']['viewCount']
        parsed_video_details['statistics']['likeCount'] = items['statistics']['likeCount']
        parsed_video_details['statistics']['commentCount'] = items['statistics']['commentCount']
        return parsed_video_details

In [219]:
find_video = FindVideo('x7X9w_GIm1s')

In [220]:
video = find_video.find_video(youtube_client)

In [221]:
FindVideo('x7X9w_GIm1s').find_video(youtube_client).get_video_details()

{'id': 'x7X9w_GIm1s',
 'channel_id': 'UCsBjURrPoezykLs9EqgamOA',
 'title': 'Python in 100 Seconds',
 'channel_title': 'Fireship',
 'description': "Python is arguably the world's most popular programming language. It is easy to learn, yet suitable in professional software like web applications, data science, and server-side scripts. https://fireship.io/tags/python/\n\n#python #programming #100SecondsOfCode\n\n🔗 Resources\n\nPython Docs https://docs.python.org/3/\nPython TIOBE Ranking https://www.infoworld.com/article/3636789/python-tops-tiobe-language-index.html\n\n🔥 Get More Content - Upgrade to PRO\n\nUpgrade to Fireship PRO at https://fireship.io/pro\nUse code lORhwXd2 for 25% off your first payment. \n\n🎨 My Editor Settings\n\n- Atom One Dark \n- vscode-icons\n- Fira Code Font\n\n🔖 Topics Covered\n\n- Is YouTube built with Python?\n- Is Python Strongly typed?\n- Python syntax and indentation rules\n- When was python released?",
 'tags': ['webdev', 'app development', 'lesson', 'tutor

In [116]:
video.get_video_details()

{'id': 'x7X9w_GIm1s',
 'channel_id': 'UCsBjURrPoezykLs9EqgamOA',
 'title': 'Python in 100 Seconds',
 'channel_title': 'Fireship',
 'description': "Python is arguably the world's most popular programming language. It is easy to learn, yet suitable in professional software like web applications, data science, and server-side scripts. https://fireship.io/tags/python/\n\n#python #programming #100SecondsOfCode\n\n🔗 Resources\n\nPython Docs https://docs.python.org/3/\nPython TIOBE Ranking https://www.infoworld.com/article/3636789/python-tops-tiobe-language-index.html\n\n🔥 Get More Content - Upgrade to PRO\n\nUpgrade to Fireship PRO at https://fireship.io/pro\nUse code lORhwXd2 for 25% off your first payment. \n\n🎨 My Editor Settings\n\n- Atom One Dark \n- vscode-icons\n- Fira Code Font\n\n🔖 Topics Covered\n\n- Is YouTube built with Python?\n- Is Python Strongly typed?\n- Python syntax and indentation rules\n- When was python released?",
 'tags': ['webdev', 'app development', 'lesson', 'tutor

In [117]:
video.get_video_stats_details()

{'details': {'id': 'x7X9w_GIm1s',
  'channel_id': 'UCsBjURrPoezykLs9EqgamOA',
  'title': 'Python in 100 Seconds',
  'channel_title': 'Fireship',
  'description': "Python is arguably the world's most popular programming language. It is easy to learn, yet suitable in professional software like web applications, data science, and server-side scripts. https://fireship.io/tags/python/\n\n#python #programming #100SecondsOfCode\n\n🔗 Resources\n\nPython Docs https://docs.python.org/3/\nPython TIOBE Ranking https://www.infoworld.com/article/3636789/python-tops-tiobe-language-index.html\n\n🔥 Get More Content - Upgrade to PRO\n\nUpgrade to Fireship PRO at https://fireship.io/pro\nUse code lORhwXd2 for 25% off your first payment. \n\n🎨 My Editor Settings\n\n- Atom One Dark \n- vscode-icons\n- Fira Code Font\n\n🔖 Topics Covered\n\n- Is YouTube built with Python?\n- Is Python Strongly typed?\n- Python syntax and indentation rules\n- When was python released?",
  'tags': ['webdev', 'app development',

In [245]:
class YouTube:
    def __init__(self, credentials_file='', token_path=''):
        self.__auth = Authenticate(credentials_file, token_path)
        self.__youtube_client = None
        self.__is_authenticated = False
        
    def is_authenticated(self):
        return self.__is_authenticated
        
    def authenticate(self, credentials_file='', token_path=''):
        self.__youtube_client = self.__auth.authenticate()
        self.__is_authenticated = True
        
    def get_youtube(self):
        return self.__youtube_client
        
    def search_video(self, query_string: str) -> list[str]:
        video_search = VideoSearch('python')
        search_results = video_search.search_video(self.__youtube_client)
        parsed_results = video_search.parse_basic_response(search_results)
        return parsed_results
    
    def find_video_by_url(self, video_url: str):
        """Get a specific video given the video url."""
        video_id = self.__get_video_id(video_url)
        return self.find_video_by_id(video_id)
        
    def find_video_by_id(self, video_id: str):
        """Find a video by id."""
        youtube_video = FindVideo(video_id).find_video(self.__youtube_client)
        return youtube_video
    
    @staticmethod
    def __get_video_id(video_url: str) -> str:
        """Get vdeo ID from video url"""
        if not video_url:
            raise ValueError('The video_ur has to be provided.')
        if not isinstance(video_url, str):
            raise TypeError('Te video_url has to be a string.')
        if '=' not in video_url:
            url_format = 'https://www.youtube.com/watch?v=Dqdu-FsBk0s'
            raise ValueError('Te video_url should be of the format "{url_format}"')
        video_url = video_url.split('=')[1]
        return video_url

In [223]:
youtube = YouTube(credentials_file='/home/lyle/Downloads/client_secret.json', 
                  token_path='')
youtube.authenticate()
youtube_client = youtube.get_youtube()

In [224]:
youtube_video = youtube.find_video('x7X9w_GIm1s')
youtube_video.get_video_stats_details()

{'details': {'id': 'x7X9w_GIm1s',
  'channel_id': 'UCsBjURrPoezykLs9EqgamOA',
  'title': 'Python in 100 Seconds',
  'channel_title': 'Fireship',
  'description': "Python is arguably the world's most popular programming language. It is easy to learn, yet suitable in professional software like web applications, data science, and server-side scripts. https://fireship.io/tags/python/\n\n#python #programming #100SecondsOfCode\n\n🔗 Resources\n\nPython Docs https://docs.python.org/3/\nPython TIOBE Ranking https://www.infoworld.com/article/3636789/python-tops-tiobe-language-index.html\n\n🔥 Get More Content - Upgrade to PRO\n\nUpgrade to Fireship PRO at https://fireship.io/pro\nUse code lORhwXd2 for 25% off your first payment. \n\n🎨 My Editor Settings\n\n- Atom One Dark \n- vscode-icons\n- Fira Code Font\n\n🔖 Topics Covered\n\n- Is YouTube built with Python?\n- Is Python Strongly typed?\n- Python syntax and indentation rules\n- When was python released?",
  'tags': ['webdev', 'app development',

In [225]:
class VideoSearch(YouTubeSearch):
    MAX_RESULTS = 5
    REGION_CODE = 'us'
    
    def __init__(self, query_string: str):
        self.__type = YouTubeSearchType.VIDEO
        self.__query = YouTubeSearchQuery(query_string)
        
    def __get_query(self):
        return self.__query.query_string
        
    def basic_info(self):
        basic_info_params = self.__generate_basic_info_params()
        return basic_info_params
    
    def advanced_info(self):
        pass
    
    def all_info(self):
        pass
    
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            part='id,snippet',
            type=self.__type,
            q=self.__get_query(),
            maxResults=self.MAX_RESULTS,
            regionCode=self.REGION_CODE
        ) 
        return basic_info_params
    
    def search_video(self, youtube_client, search_type='basic'):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params()
            search_request = youtube_client.search().list(
                **basic_info_params
            )
            search_response = search_request.execute()
        return search_response
    
    def __parse_basic_response(self, search_result: dict, youtube_client) -> list[dict[str, str]]:
        video_details = []
        video_results = search_result['items']
        for video_result in video_results:
            video_id = video_result['id']
            youtube_video = FindVideo(video_id).find_video(youtube_client)
            video_details.append(youtube_video)
        return video_details

In [226]:
videos = VideoSearch('python programming').search_video(youtube_client)

In [227]:
videos

{'kind': 'youtube#searchListResponse',
 'etag': 'UHVGeK_xdVTxJvgmfKb6N5mK_fQ',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'US',
 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'ulOXFQl7L257T5mztOUvrLY4UXU',
   'id': {'kind': 'youtube#video', 'videoId': 'kqtD5dpn9C8'},
   'snippet': {'publishedAt': '2020-09-16T13:00:20Z',
    'channelId': 'UCWv7vMbMWH4-V0ZXdmDpPBA',
    'title': 'Python for Beginners - Learn Python in 1 Hour',
    'description': 'This Python tutorial for beginners show how to get started with Python quickly. Learn to code in 1 hour! Watch this tutorial get ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/kqtD5dpn9C8/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/kqtD5dpn9C8/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/kqtD5dpn9C8/hqdefault.jpg',
      'width': 480,
      'h

In [236]:
def parse_videos(search_response, youtube_client):
    videos = []
    video_results = search_response['items']
    for video_result in video_results:
        video_id = video_result['id']['videoId']
        youtube_video = FindVideo(video_id).find_video(youtube_client)
        videos.append(youtube_video)
    return videos

In [237]:
videos = parse_videos(videos, youtube_client)
videos

In [238]:
videos[0].get_video_stats_details()

{'details': {'id': 'kqtD5dpn9C8',
  'channel_id': 'UCWv7vMbMWH4-V0ZXdmDpPBA',
  'title': 'Python for Beginners - Learn Python in 1 Hour',
  'channel_title': 'Programming with Mosh',
  'description': 'This Python tutorial for beginners show how to get started with Python quickly. Learn to code in 1 hour! Watch this tutorial get started! \n👍 Subscribe for more Python tutorials like this: https://goo.gl/6PYaGF\n🔥 Want to learn more? Watch my complete Python course: https://youtu.be/_uQrJ0TkZlc\n\n📕 Get my FREE Python cheat sheet: http://bit.ly/2Gp80s6\n\nCourses: https://codewithmosh.com\nTwitter: https://twitter.com/moshhamedani\nFacebook: https://www.facebook.com/programmingwithmosh/\nBlog: http://programmingwithmosh.com\n\n#Python, #MachineLearning, #WebDevelopment\n\n📔 Python Exercises for Beginners: https://goo.gl/1XnQB1\n\n⭐ My Favorite Python Books\n- Python Crash Course: https://amzn.to/2GqMdjG\n- Automate the Boring Stuff with Python: https://amzn.to/2N71d6S\n- A Smarter Way to L

In [243]:
class VideoSearch(YouTubeSearch):
    MAX_RESULTS = 5
    REGION_CODE = 'us'
    
    def __init__(self, query_string: str):
        self.__type = YouTubeSearchType.VIDEO
        self.__query = YouTubeSearchQuery(query_string)
        
    def __get_query(self):
        return self.__query.query_string
        
    def basic_info(self):
        basic_info_params = self.__generate_basic_info_params()
        return basic_info_params
    
    def advanced_info(self):
        pass
    
    def all_info(self):
        pass
    
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            part='id,snippet',
            type=self.__type,
            q=self.__get_query(),
            maxResults=self.MAX_RESULTS,
            regionCode=self.REGION_CODE
        ) 
        return basic_info_params
    
    def search_video(self, youtube_client, search_type='basic'):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params()
            search_request = youtube_client.search().list(
                **basic_info_params
            )
            search_response = search_request.execute()
            videos = self.__parse_basic_response(search_response, youtube_client)
        return videos
    
    def __parse_basic_response(self, search_response, youtube_client):
        videos = []
        video_results = search_response['items']
        for video_result in video_results:
            video_id = video_result['id']['videoId']
            youtube_video = FindVideo(video_id).find_video(youtube_client)
            videos.append(youtube_video)
        return videos

In [244]:
videos = VideoSearch('python programming').search_video(youtube_client)
videos

In [248]:
class YouTube:
    def __init__(self, credentials_file='', token_path=''):
        self.__auth = Authenticate(credentials_file, token_path)
        self.__youtube_client = None
        self.__is_authenticated = False
        
    def is_authenticated(self):
        return self.__is_authenticated
        
    def authenticate(self, credentials_file='', token_path=''):
        self.__youtube_client = self.__auth.authenticate()
        self.__is_authenticated = True
        
    def get_youtube(self):
        return self.__youtube_client
        
    def search_video(self, query_string: str) -> list[str]:
        videos = VideoSearch(query_string).search_video(self.__youtube_client)
        return videos
    
    def find_video_by_url(self, video_url: str):
        """Get a specific video given the video url."""
        video_id = self.__get_video_id(video_url)
        return self.find_video_by_id(video_id)
        
    def find_video_by_id(self, video_id: str):
        """Find a video by id."""
        youtube_video = FindVideo(video_id).find_video(self.__youtube_client)
        return youtube_video
    
    @staticmethod
    def __get_video_id(video_url: str) -> str:
        """Get vdeo ID from video url"""
        if not video_url:
            raise ValueError('The video_ur has to be provided.')
        if not isinstance(video_url, str):
            raise TypeError('Te video_url has to be a string.')
        if '=' not in video_url:
            url_format = 'https://www.youtube.com/watch?v=Dqdu-FsBk0s'
            raise ValueError('Te video_url should be of the format "{url_format}"')
        video_url = video_url.split('=')[1]
        return video_url

In [249]:
youtube = YouTube(credentials_file='/home/lyle/Downloads/client_secret.json', 
                  token_path='')
youtube.authenticate()
youtube_client = youtube.get_youtube()

In [251]:
videos = youtube.search_video('python programming')

In [252]:
videos

In [253]:
videos[0].get_video_stats_details()

{'details': {'id': 'kqtD5dpn9C8',
  'channel_id': 'UCWv7vMbMWH4-V0ZXdmDpPBA',
  'title': 'Python for Beginners - Learn Python in 1 Hour',
  'channel_title': 'Programming with Mosh',
  'description': 'This Python tutorial for beginners show how to get started with Python quickly. Learn to code in 1 hour! Watch this tutorial get started! \n👍 Subscribe for more Python tutorials like this: https://goo.gl/6PYaGF\n🔥 Want to learn more? Watch my complete Python course: https://youtu.be/_uQrJ0TkZlc\n\n📕 Get my FREE Python cheat sheet: http://bit.ly/2Gp80s6\n\nCourses: https://codewithmosh.com\nTwitter: https://twitter.com/moshhamedani\nFacebook: https://www.facebook.com/programmingwithmosh/\nBlog: http://programmingwithmosh.com\n\n#Python, #MachineLearning, #WebDevelopment\n\n📔 Python Exercises for Beginners: https://goo.gl/1XnQB1\n\n⭐ My Favorite Python Books\n- Python Crash Course: https://amzn.to/2GqMdjG\n- Automate the Boring Stuff with Python: https://amzn.to/2N71d6S\n- A Smarter Way to L

In [ ]:
class FindVideo:
    def __init__(self, video_id: str):
        """Find the video with the given id."""
        self.__video_id = video_id
        
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            id=self.__video_id,
            part='snippet,contentDetails,statistics'
        ) 
        return basic_info_params
    
    def find_video(self, youtube_client):
        """Find the video."""
        basic_info_params = self.__generate_basic_info_params()
        search_request = youtube_client.videos().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        parsed_response = self.__parse_video_details(search_response)
        youtube_video = YouTubeVideo(parsed_response)
        return youtube_video
    
    def __parse_video_details(self, video_details: dict):
        """Parse the video details.

        Returns
        -------
        parsed_video_details: dict
            A dictionary of the YouTube video details.
        """
        parsed_video_details = dict()
        items = video_details['items'][0]
        parsed_video_details['details'] = dict()
        parsed_video_details['statistics'] = dict()
        parsed_video_details['details']['id'] = items['id']
        parsed_video_details['details']['channelId'] = items['snippet']['channelId']
        parsed_video_details['details']['title'] = items['snippet']['title']
        parsed_video_details['details']['channelTitle'] = items['snippet']['channelTitle']
        parsed_video_details['details']['description'] = items['snippet']['description']
        parsed_video_details['details']['tags'] = items['snippet']['tags']
        parsed_video_details['details']['duration'] = items['contentDetails']['duration']
        parsed_video_details['details']['licensedContent'] = items['contentDetails']['licensedContent']
        parsed_video_details['statistics']['viewCount'] = items['statistics']['viewCount']
        parsed_video_details['statistics']['likeCount'] = items['statistics']['likeCount']
        parsed_video_details['statistics']['commentCount'] = items['statistics']['commentCount']
        return parsed_video_details

In [254]:
search_request = youtube_client.commentThreads().list(
    videoId='rfscVS0vtbw',
    part='snippet,replies'
)
search_response = search_request.execute()

In [255]:
search_response

{'kind': 'youtube#commentThreadListResponse',
 'etag': 'GjW7_YjyIqpPKzd8L87El9oU_J0',
 'nextPageToken': 'QURTSl9pMFRzUDBEUVE3OWNYZ29KUUNmc2R6NXJYLThaZEJKQXc4WXAwNzF1SUY2OURxLXMtNEZfcl9LTDdsZDRVcTFrQ1dPQlkzQUFyWU1lZFNWZ3hibkVYMGQ1dEMySkE=',
 'pageInfo': {'totalResults': 20, 'resultsPerPage': 20},
 'items': [{'kind': 'youtube#commentThread',
   'etag': '6yra7o-RE4sCbwE_uomh3f7EhGg',
   'id': 'Ugxqcek3RUzxWv-4VqB4AaABAg',
   'snippet': {'videoId': 'rfscVS0vtbw',
    'topLevelComment': {'kind': 'youtube#comment',
     'etag': 'kHla154smqfAXxy9m5RKxmAita8',
     'id': 'Ugxqcek3RUzxWv-4VqB4AaABAg',
     'snippet': {'videoId': 'rfscVS0vtbw',
      'textDisplay': 'Want more from Mike? He&#39;s starting a coding RPG/Bootcamp - <a href="https://simulator.dev/">https://simulator.dev/</a>',
      'textOriginal': "Want more from Mike? He's starting a coding RPG/Bootcamp - https://simulator.dev/",
      'authorDisplayName': 'freeCodeCamp.org',
      'authorProfileImageUrl': 'https://yt3.ggpht.com/yt

In [257]:
parse_comments(search_response)

[{'kind': 'youtube#commentThread',
  'etag': '6yra7o-RE4sCbwE_uomh3f7EhGg',
  'id': 'Ugxqcek3RUzxWv-4VqB4AaABAg',
  'snippet': {'videoId': 'rfscVS0vtbw',
   'topLevelComment': {'kind': 'youtube#comment',
    'etag': 'kHla154smqfAXxy9m5RKxmAita8',
    'id': 'Ugxqcek3RUzxWv-4VqB4AaABAg',
    'snippet': {'videoId': 'rfscVS0vtbw',
     'textDisplay': 'Want more from Mike? He&#39;s starting a coding RPG/Bootcamp - <a href="https://simulator.dev/">https://simulator.dev/</a>',
     'textOriginal': "Want more from Mike? He's starting a coding RPG/Bootcamp - https://simulator.dev/",
     'authorDisplayName': 'freeCodeCamp.org',
     'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AL5GRJXPR4dSz0wwP-elkeiWUMnfZlCtNZP8Rd-tmFOZYg=s48-c-k-c0x00ffffff-no-rj',
     'authorChannelUrl': 'http://www.youtube.com/channel/UC8butISFwT-Wl7EV0hUK0BQ',
     'authorChannelId': {'value': 'UC8butISFwT-Wl7EV0hUK0BQ'},
     'canRate': True,
     'viewerRating': 'none',
     'likeCount': 3389,
     'publishedAt':

In [ ]:
'updatedAt': item['snippet']['topLevelComment']['snippet']['updatedAt'],
                'publishedAt': item['snippet']['topLevelComment']['snippet']['publishedAt'],
                'likeCount': item['snippet']['topLevelComment']['snippet']['likeCount']

In [291]:
def parse_comments(search_response):
    items = search_response['items']
    comments = []
    for item in items:
        comments.append({
            'id': item['id'],
            'videoId': item['snippet']['videoId'],
            'totalReplyCount': item['snippet']['totalReplyCount'],
            'topLevelComment': {
                'id': item['snippet']['topLevelComment']['id'],
                'textDisplay': item['snippet']['topLevelComment']['snippet']['textDisplay'],
                'textOriginal': item['snippet']['topLevelComment']['snippet']['textOriginal'],
                'authorDisplayName': item['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                'authorProfileImageUrl': item['snippet']['topLevelComment']['snippet']['authorProfileImageUrl'],
                'authorChannelId': item['snippet']['topLevelComment']['snippet']['authorChannelId']['value'],
                'likeCount': item['snippet']['topLevelComment']['snippet']['likeCount'],
                'publishedAt': item['snippet']['topLevelComment']['snippet']['publishedAt'],
                'updatedAt': item['snippet']['topLevelComment']['snippet']['updatedAt']
            }
        })
    return comments

In [292]:
parse_comments(search_response)

[{'id': 'Ugxqcek3RUzxWv-4VqB4AaABAg',
  'videoId': 'rfscVS0vtbw',
  'totalReplyCount': 149,
  'topLevelComment': {'id': 'Ugxqcek3RUzxWv-4VqB4AaABAg',
   'textDisplay': 'Want more from Mike? He&#39;s starting a coding RPG/Bootcamp - <a href="https://simulator.dev/">https://simulator.dev/</a>',
   'textOriginal': "Want more from Mike? He's starting a coding RPG/Bootcamp - https://simulator.dev/",
   'authorDisplayName': 'freeCodeCamp.org',
   'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AL5GRJXPR4dSz0wwP-elkeiWUMnfZlCtNZP8Rd-tmFOZYg=s48-c-k-c0x00ffffff-no-rj',
   'authorChannelId': 'UC8butISFwT-Wl7EV0hUK0BQ',
   'likeCount': 3389,
   'publishedAt': '2021-03-09T18:41:00Z',
   'updatedAt': '2021-03-09T18:41:00Z'}},
 {'id': 'UgypvMnIQz3q2eZcjY94AaABAg',
  'videoId': 'rfscVS0vtbw',
  'totalReplyCount': 0,
  'topLevelComment': {'id': 'UgypvMnIQz3q2eZcjY94AaABAg',
   'textDisplay': 'lists see again',
   'textOriginal': 'lists see again',
   'authorDisplayName': 'mystrey sinhala',
   'au

In [ ]:
{'kind': 'youtube#commentThread',
  'etag': 'szCywO9071pY_J1GEhXfyD_aI2A',
  'id': 'Ugy64XZSRWXDg_8haB94AaABAg',
  'snippet': {'videoId': 'rfscVS0vtbw',
   'topLevelComment': {'kind': 'youtube#comment',
    'etag': 'cI06zBZ48OgWVSN1E9DaYpIPyEI',
    'id': 'Ugy64XZSRWXDg_8haB94AaABAg',
    'snippet': {'videoId': 'rfscVS0vtbw',
     'textDisplay': 'how do i use pip command in terminal?',
     'textOriginal': 'how do i use pip command in terminal?',
     'authorDisplayName': 'Arsh Sahu',
     'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AL5GRJUUn74ydwoFB7kar-uFv4tbgNNTvpLoMftNTVekKA=s48-c-k-c0x00ffffff-no-rj',
     'authorChannelUrl': 'http://www.youtube.com/channel/UCiBgZ6E5os7RpYYarHHQRfg',
     'authorChannelId': {'value': 'UCiBgZ6E5os7RpYYarHHQRfg'},
     'canRate': True,
     'viewerRating': 'none',
     'likeCount': 0,
     'publishedAt': '2023-03-06T16:25:45Z',
     'updatedAt': '2023-03-06T16:25:45Z'}},
   'canReply': True,
   'totalReplyCount': 0,
   'isPublic': True}}